In [7]:
!pip install pandas numpy matplotlib seaborn 



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install joblib scikit-learn


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.7 MB 4.0 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.7 MB 3.9 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/8.7 MB 3.7 MB/s eta 0:00:02
   -------------- ------------------------- 3.1/8.7 MB 3.6 MB/s eta 0:00:02
   ------------------ --------------------- 3.9/8.7 MB 3.6 MB/s eta 0:00:02
   --------------------- ------------------ 4.7/8.7 MB 3.6 MB/s eta 0:00:02
   ------------------------- -------------- 5.5/8.7 MB 3.6 MB/s eta 0:00:01
   ---------------------------- ----------- 6.3/8.7 MB 3.6 MB/s eta 0:00:01
   -------------------------------- ------- 7.1/8.7 MB 3.7 MB/s eta 0:00:01
   ------------------------------------- -- 8.1/8.7 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.4 MB ? eta -:--:--
    ---------------------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


Matplotlib is building the font cache; this may take a moment.


In [3]:
!pip install openpyxl



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

df = pd.read_excel("ev data set.xlsx")



In [5]:
df.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,September 30 2022,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,December 31 2022,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,January 31 2020,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,June 30 2022,Ferry,WA,Truck,0,0,0,3575,3575,0.00
4,July 31 2021,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [7]:
df.shape

(20819, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20819 entries, 0 to 20818
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date                                      20819 non-null  object 
 1   County                                    20733 non-null  object 
 2   State                                     20733 non-null  object 
 3   Vehicle Primary Use                       20819 non-null  object 
 4   Battery Electric Vehicles (BEVs)          20819 non-null  int64  
 5   Plug-In Hybrid Electric Vehicles (PHEVs)  20819 non-null  int64  
 6   Electric Vehicle (EV) Total               20819 non-null  int64  
 7   Non-Electric Vehicle Total                20819 non-null  int64  
 8   Total Vehicles                            20819 non-null  int64  
 9   Percent Electric Vehicles                 20819 non-null  float64
dtypes: float64(1), int64(5), object(4)

In [9]:
df.isnull()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
20814,False,False,False,False,False,False,False,False,False,False
20815,False,False,False,False,False,False,False,False,False,False
20816,False,False,False,False,False,False,False,False,False,False
20817,False,False,False,False,False,False,False,False,False,False


In [10]:
df.isnull().sum()

Date                                         0
County                                      86
State                                       86
Vehicle Primary Use                          0
Battery Electric Vehicles (BEVs)             0
Plug-In Hybrid Electric Vehicles (PHEVs)     0
Electric Vehicle (EV) Total                  0
Non-Electric Vehicle Total                   0
Total Vehicles                               0
Percent Electric Vehicles                    0
dtype: int64

In [12]:
# Step 1: Convert column to numeric (safely)
df['Percent Electric Vehicles'] = pd.to_numeric(df['Percent Electric Vehicles'], errors='coerce')

# Step 2: Drop rows where conversion failed (optional, if any)
df = df.dropna(subset=['Percent Electric Vehicles'])

# Step 3: Calculate IQR bounds
Q1 = df['Percent Electric Vehicles'].quantile(0.25)
Q3 = df['Percent Electric Vehicles'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print('Lower Bound:', lower_bound)
print('Upper Bound:', upper_bound)

# Step 4: Detect outliers
outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | 
              (df['Percent Electric Vehicles'] > upper_bound)]

print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])


Lower Bound: -3.5174999999999996
Upper Bound: 6.9025
Number of outliers in 'Percent Electric Vehicles': 2476


In [13]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Remove rows where 'Date' or 'Electric Vehicle (EV) Total' is missing
df = df[df['Date'].notnull()]
df = df[df['Electric Vehicle (EV) Total'].notnull()]

# Fill missing values in 'County' and 'State' columns
df['County'] = df['County'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')

print("Missing after fill:")
print(df[['County','State']].isnull().sum())

df.head()


Missing after fill:
County    0
State     0
dtype: int64


,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,2022-09-30,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,2022-12-31,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,2020-01-31,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,2022-06-30,Ferry,WA,Truck,0,0,0,3575,3575,0.00
4,2021-07-31,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [15]:
import numpy as np


In [16]:
df['Percent Electric Vehicles'] = np.where(
    df['Percent Electric Vehicles'] > upper_bound,
    upper_bound,
    np.where(
        df['Percent Electric Vehicles'] < lower_bound,
        lower_bound,
        df['Percent Electric Vehicles']
    )
)

outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | (df['Percent Electric Vehicles'] > upper_bound)] 
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])


Number of outliers in 'Percent Electric Vehicles': 0


In [17]:
pip install numpy pandas openpyxl matplotlib seaborn scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
